In [1]:
import keras_forward as kf
import tf_forward as tff
import keras_models as kms
import tf_graphs as tfgs
import numpy as np

/home/astrophysics/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_inputs = 28 * 28
num_outputs = 10
m = 1000
batch_size = 1000
a1_size = 5

km = kms.slp_model(num_inputs, num_outputs)
km.compile(loss='mse', optimizer='rmsprop')
np.random.seed(1337)
x_tr = np.random.random((m, num_inputs))
y_tr = np.random.randint(size=(m, num_outputs), low = 0, high = 2)
num_weights = (num_inputs + 1) * a1_size + (a1_size + 1) * num_outputs

km = kf.keras_model(km, x_tr, y_tr, batch_size) 
km.setup_LL()

tf_graph = tfgs.slp_graph
layer_sizes = [a1_size]
tfm = tff.tf_model(tf_graph, x_tr, y_tr, batch_size, layer_sizes)
fit_metric = 'chisq'
tfm.setup_LL(fit_metric)

n = 1
weights = np.random.random((n, num_weights))


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
%%timeit
for i in range(n):
    tfm(weights[i])

10 loops, best of 3: 20.6 ms per loop


In [4]:
%%timeit
for i in range(n):
    km(weights[i])

1000/1000 [==============================]1000/1000 [==============================] - 0s 62us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 63us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 61us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 68us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 65us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 64us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 66us/step

1000/1000 [==============================]1000/1000 [==============================] - ETA:  - 0s 49us/step

1000/1000 [==============================]1000/1000 [==============================] - 0s 67us/step 

1000/1000 [==============================]1000/1000 [=============================

In [5]:
import pstats, cProfile

In [8]:
cProfile.runctx("km(weights[0])", globals(), locals(), "kmProfile.prof")

s = pstats.Stats("kmProfile.prof")
s.strip_dirs().sort_stats("time").print_stats()


1000/1000 [==============================]1000/1000 [==============================] - 0s 137us/step

Thu Sep 20 16:17:59 2018    kmProfile.prof

         16477 function calls (16409 primitive calls) in 0.164 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       33    0.030    0.001    0.030    0.001 {_pywrap_tensorflow_internal.TF_Run}
       34    0.027    0.001    0.027    0.001 socket.py:333(send)
       33    0.018    0.001    0.111    0.003 session.py:1042(_run)
       33    0.010    0.000    0.021    0.001 session.py:407(__init__)
       32    0.009    0.000    0.010    0.000 training.py:383(_slice_arrays)
     2998    0.005    0.000    0.005    0.000 {isinstance}
      604    0.003    0.000    0.003    0.000 ops.py:1753(name)
      236    0.003    0.000    0.005    0.000 ops.py:322(name)
        1    0.003    0.003    0.144    0.144 training.py:1324(_test_loop)
      104    0.003    0.000    0.003    0.000 {numpy.c

<pstats.Stats instance at 0x7f3bdc6c2368>

In [7]:
cProfile.runctx("tfm(weights[0])", globals(), locals(), "tfmProfile.prof")

s = pstats.Stats("tfmProfile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Thu Sep 20 16:17:30 2018    tfmProfile.prof

         1271 function calls in 0.068 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.025    0.025    0.025    0.025 {_pywrap_tensorflow_internal.TF_Run}
      128    0.009    0.000    0.009    0.000 {isinstance}
       34    0.007    0.000    0.007    0.000 ops.py:564(__hash__)
        1    0.007    0.007    0.007    0.007 {method 'SerializeToString' of 'google.protobuf.pyext._message.CMessage' objects}
        1    0.006    0.006    0.006    0.006 {_pywrap_tensorflow_internal.TF_ExtendGraph}
      106    0.005    0.000    0.005    0.000 {method 'ByteSize' of 'google.protobuf.pyext._message.CMessage' objects}
      106    0.002    0.000    0.002    0.000 {method 'extend' of 'google.protobuf.pyext._message.RepeatedCompositeContainer' objects}
        1    0.001    0.001    0.009    0.009 ops.py:2925(_as_graph_def)
        1    0.001    0.001    0.001    0.001 {_pyw

<pstats.Stats instance at 0x7f3bdc77d6c8>